In [1]:
%load_ext watermark
%watermark

2022-02-21T18:32:19+01:00

CPython 3.6.7
IPython 6.4.0

compiler   : GCC 7.3.0
system     : Linux
release    : 4.15.0-167-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 8
interpreter: 64bit


In [2]:
# %pylab

import numpy
import matplotlib
from matplotlib import pylab, mlab, pyplot
from mpl_toolkits.mplot3d import Axes3D
np = numpy
plt = pyplot

from pylab import *
from numpy import *

In [3]:
from scipy.optimize import curve_fit

import tkinter as tk
from tkinter import filedialog
%matplotlib qt

In [4]:
def load_file_GUI(dir_string):

    root = tk.Tk()
    root.withdraw()

    file_path = filedialog.askopenfilename(initialdir = dir_string,
                                       multiple=True)
    return file_path

def beta_continue_alamano(a,q,beta_guess): # a l'air ok
    C1 = q**2/( (beta_guess+2)**2 - a - q**2/( (beta_guess+4)**2 - a - q**2/( (beta_guess+6)**2 - a - q**2/( (beta_guess+8)**2 - a - q**2/( (beta_guess+10)**2 - a ) ) ) ) )
    C2 = q**2/( (beta_guess-2)**2 - a - q**2/( (beta_guess-4)**2 - a - q**2/( (beta_guess-6)**2 - a - q**2/( (beta_guess-8)**2 - a - q**2/( (beta_guess-10)**2 - a  ) ) ) ) )
    return a + C1 + C2

In [6]:
# Constantes de la physique
# ABSOLUMENT RECHARGER APRÈS AVOIR EXECUTÉ LES CASES D'IMPORT AU DESSUS

C_e = 1.602e-19        # Coulomb
kb = 1.38064852*1e-23  # Boltzman
m_Ca = 40.078*1.66054e-27 # masse Ca 40.078
m_GM = 1e6*1.66054e-27 # la masse de la GMol
eps0 = 8.854187*1e-12  # permittivité électrique du vide

r0 = 2.5e-3 # 2.5e-3   # rayon piège Ca+
d0 = 4e-3/2            # longueur piège Ca+

Omega = 2.05e6*2*pi # 2.047e6

bk = 4 # nombre de barreaux par groupe (2 -> 4-pole , 4 -> 8-pole ...)

mkappa = 0.23          # écrantage piège réel GiantMol

wzLC = (2*pi*90806.9982303)**2
kappa_simion = m_Ca*d0**2*wzLC/(2*C_e)
print('%s = %f' % ('$\kappa_{simion}$',kappa_simion) )

zeta = kappa_simion*r0**2/d0**2

$\kappa_{simion}$ = 0.270471


In [7]:
# beta non adiabatique
# for a given q_x, compute Urf, beta_x, omega_x
# and Udc for spherical cloud alpha=1
a = 0
q = linspace(0,1,11)
beta_adia = sqrt(a+q**2/2)
beta_nonadia = np.zeros((len(beta_adia)))
print('  q ', '    beta non-a', ' beta a')
for j,k in enumerate(q):
    for l in range(15):
        beta_nonadia[j] = beta_continue_alamano(a,k,beta_nonadia[j])
        beta_nonadia[j] = sqrt(beta_nonadia[j])
    print('>',f'{k:.02f}','->',f'{beta_nonadia[j]:.03f}',' ~ ',f'{beta_adia[j]:.03f}')
print('')

print('|    q   |     Urf     |    beta    |      fx      |     Udc     |')
print('|:--------:|:-------------:|:------------:|:--------------:|:-------------:|')
omega_nonadia = np.zeros((len(beta_adia)))    
Udc_for_sphere = np.zeros((len(beta_adia)))
for i,j in enumerate(beta_nonadia):
    omega_nonadia[i]  = 0.5*Omega*j
    Udc_for_sphere[i] = m_Ca*d0**2/(2*kappa_simion*C_e)*omega_nonadia[i]**2
    Urf = q[i]*m_Ca*r0**2*Omega**2/(4*C_e)
    print(f'| {q[i]:.02f} | {Urf:7.03f} | {j:.03f} | {omega_nonadia[i]/2/np.pi:10.03f} | {Udc_for_sphere[i]:07.03f} |')

  q      beta non-a  beta a
> 0.00 -> 0.000  ~  0.000
> 0.10 -> 0.071  ~  0.071
> 0.20 -> 0.143  ~  0.141
> 0.30 -> 0.216  ~  0.212
> 0.40 -> 0.293  ~  0.283
> 0.50 -> 0.374  ~  0.354
> 0.60 -> 0.462  ~  0.424
> 0.70 -> 0.563  ~  0.495
> 0.80 -> 0.689  ~  0.566
> 0.90 -> 0.912  ~  0.636
> 1.00 -> nan  ~  0.707

|    q   |     Urf     |    beta    |      fx      |     Udc     |
|:--------:|:-------------:|:------------:|:--------------:|:-------------:|
| 0.00 |   0.000 | 0.000 |      0.000 | 000.000 |
| 0.10 |  10.769 | 0.071 |  72620.791 | 000.640 |
| 0.20 |  21.538 | 0.143 | 146115.110 | 002.589 |
| 0.30 |  32.307 | 0.216 | 221460.613 | 005.948 |
| 0.40 |  43.076 | 0.293 | 299880.367 | 010.906 |
| 0.50 |  53.846 | 0.374 | 383087.725 | 017.797 |
| 0.60 |  64.615 | 0.462 | 473805.699 | 027.225 |
| 0.70 |  75.384 | 0.563 | 577142.815 | 040.395 |
| 0.80 |  86.153 | 0.689 | 705777.832 | 060.408 |
| 0.90 |  96.922 | 0.912 | 934751.214 | 105.963 |
| 1.00 | 107.691 | nan |        nan | 0000n

/home/adrian/.local/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in sqrt
  if sys.path[0] == '':


In [5]:
%run ../../GiantMol/Functions/1.2/data_fct_Adrien_point_by_point
matplotlib.rcParams.update({'font.size': 25})

In [7]:
def data_retrieve(all_subdir,points_and_coord, condition_parameters, slash_cfg,**kwargs):

    myslashpos = slash_cfg[0]
    slashcond = slash_cfg[1]

    # determining number of elements on each repetition
    try:
        num_runs = ['Try'+str(kwargs.get('forcenumtry'))]
        num_runs_aux = [runs[myslashpos[slashcond+1]+1:] for runs in all_subdir if list(points_and_coord.keys())[0] in runs]
        num_runs_aux = list(dict.fromkeys(num_runs_aux))
        num_runs_aux = len(num_runs_aux)
        all_subdir = [x for w,x in enumerate(all_subdir) if (w-kwargs.get('forcenumtry'))%num_runs_aux==0]
    except:
        num_runs = [runs[myslashpos[slashcond+1]+1:] for runs in all_subdir if list(points_and_coord.keys())[0] in runs]
        num_runs = list(dict.fromkeys(num_runs))

    # number of repetitions
    print('> Points |',len(points_and_coord))
    print('> Simulations pour chaque point |', num_runs)
    
    data0 = [[] for i in range(len(points_and_coord))] # file path to SimuType4
    data_address = [[] for i in range(len(points_and_coord))]

    # Variables à deux coordonnées : [point, try]
    shapevar = (len(points_and_coord),len(num_runs))
    Time        = []
    Temperature = []
    
    t0 = time.clock()
    print("Hello")

    # write variables
    for k, address in enumerate(all_subdir):

    # in-loop variables
        pnt = k // len(num_runs)  # actual point
        rep = k  % len(num_runs)          # actual repetition

        # get only .dat files in each simulation directory
        onlyfiles = [f for f in listdir(address) if isfile(join(address, f)) and not "xva" in f and ".dat" in f]
        # build path file
        data0[pnt].append('{}/{}'.format(address,sort(onlyfiles)[0].strip('.dat')))
        data_address[pnt].append(address)

        # load fluorescence and T
        Time.append( loadtxt(str(data0[pnt][0])+'.dat',unpack=True)[0] )
        Temperature.append( loadtxt(str(data0[pnt][0])+'.dat',unpack=True)[4:7] )
        
        # load cloud size before injection
        if not(rep % len(num_runs)):
            print( "Point n°", pnt )
        
        print(f'{pnt:02}','-',f'{rep:02}',' > ',data0[pnt][rep])
        
        if k == len(points_and_coord)*len(num_runs) - 1:
            break

    t1 = time.clock() - t0
    print("Time elapsed: ", t1, 's') # CPU seconds elapsed (floating point)
    print("Time elapsed: ", t1/60, 'm') # CPU seconds elapsed (floating point)
    
    data_name = [data_address, data0]
    
    return data_name, num_runs, Time, Temperature 

# Extract data as npz

In [7]:
file_path = load_file_GUI('/home/adrian/Documents/Programmes/Python/GiantMol/SimAnalysisPointbyPoint')
print(file_path)

('/home/adrian/RemoteFS/Rivendel/Simulations/20211201/DC11_RF00/Try00/Temp_SimuType0_N01024_Vrf0060_Udc0.7000D+01V_nt1000.dat',)


In [8]:
## GUI for data loading
# Select one data file all the way down to the directories
# SELECT Temp_SimuTypeQ_N ... .dat

file_cfg, slash_cfg, all_subdir = load_gui(filter_nocomplete=1)

file_path = file_cfg[0]
dir_path = file_cfg[1]
work_rep = file_cfg[2]
filename = file_cfg[3]

myslashpos = slash_cfg[0]
slashcond = slash_cfg[1]

> Répertoire : /home/adrian/RemoteFS/Rivendel/Simulations/20211201/DC11_RF00/Try00
> Filename : Temp_SimuType0_N01024_Vrf0060_Udc0.7000D+01V_nt1000.dat
> myslashpos | [0, 5, 12, 21, 30, 42, 51, 61]
> slashcond | 6
Points deleted because they were not complete ['DC11_RF12']   2 pt(s)
Total number of data directories 24


In [9]:
## Condition for data
points_and_coord, condition_parameters = simu_conditions(all_subdir, myslashpos, slashcond, filename)

['DC11_RF00', 'DC11_RF01', 'DC11_RF02', 'DC11_RF03', 'DC11_RF04', 'DC11_RF05', 'DC11_RF06', 'DC11_RF07', 'DC11_RF08', 'DC11_RF11', 'DC11_RF13', 'DC11_RF14', 'DC11_RF15', 'DC11_RF16', 'DC11_RF17', 'DC11_RF18', 'DC11_RF19', 'DC11_RF20', 'DC11_RF21', 'DC11_RF22', 'DC11_RF23', 'DC11_RF24', 'DC11_RF25', 'DC11_RF26']
000 > DC11_RF00
001 > DC11_RF01
002 > DC11_RF02
003 > DC11_RF03
004 > DC11_RF04
005 > DC11_RF05
006 > DC11_RF06
007 > DC11_RF07
008 > DC11_RF08
009 > DC11_RF11
010 > DC11_RF13
011 > DC11_RF14
012 > DC11_RF15
013 > DC11_RF16
014 > DC11_RF17
015 > DC11_RF18
016 > DC11_RF19
017 > DC11_RF20
018 > DC11_RF21
019 > DC11_RF22
020 > DC11_RF23
021 > DC11_RF24
022 > DC11_RF25
023 > DC11_RF26
> condition names ['DC', 'RF']
> number of points 24
> N_ions = 1024
> e_GMol = 50


In [60]:
## Data loading (1D arrays)
data_name, num_runs, Time, Temperature \
= data_retrieve(all_subdir,points_and_coord,condition_parameters,slash_cfg) # stopping_power  ,forcenumtry=28

> Points | 24
> Simulations pour chaque point | ['Try00']
Hello
Point n° 0
00 - 00  >  /home/adrian/RemoteFS/Rivendel/Simulations/20211201/DC11_RF00/Try00/Temp_SimuType0_N01024_Vrf0060_Udc0.7000D+01V_nt1000
Point n° 1
01 - 00  >  /home/adrian/RemoteFS/Rivendel/Simulations/20211201/DC11_RF01/Try00/Temp_SimuType0_N01024_Vrf0061_Udc0.7000D+01V_nt1000
Point n° 2
02 - 00  >  /home/adrian/RemoteFS/Rivendel/Simulations/20211201/DC11_RF02/Try00/Temp_SimuType0_N01024_Vrf0062_Udc0.7000D+01V_nt1000
Point n° 3
03 - 00  >  /home/adrian/RemoteFS/Rivendel/Simulations/20211201/DC11_RF03/Try00/Temp_SimuType0_N01024_Vrf0063_Udc0.7000D+01V_nt1000
Point n° 4
04 - 00  >  /home/adrian/RemoteFS/Rivendel/Simulations/20211201/DC11_RF04/Try00/Temp_SimuType0_N01024_Vrf0064_Udc0.7000D+01V_nt1000
Point n° 5
05 - 00  >  /home/adrian/RemoteFS/Rivendel/Simulations/20211201/DC11_RF05/Try00/Temp_SimuType0_N01024_Vrf0065_Udc0.7000D+01V_nt1000
Point n° 6
06 - 00  >  /home/adrian/RemoteFS/Rivendel/Simulations/20211201/DC1

In [75]:
for j,k in enumerate(data_name[0]):
    cond = k[0][52:61]
    print(cond)
    outfile = 'Time_and_temp_RFHEAT_'+str(cond)
    np.savez( outfile, time=Time[j], temp = mean(Temperature[j],axis=0)) # dirname_hc=re.sub(r'\\',r'/',dirname_hc)

DC11_RF00
DC11_RF01
DC11_RF02
DC11_RF03
DC11_RF04
DC11_RF05
DC11_RF06
DC11_RF07
DC11_RF08
DC11_RF11
DC11_RF13
DC11_RF14
DC11_RF15
DC11_RF16
DC11_RF17
DC11_RF18
DC11_RF19
DC11_RF20
DC11_RF21
DC11_RF22
DC11_RF23
DC11_RF24
DC11_RF25
DC11_RF26


In [74]:
j = 17
print(Urf[j])
mean(Temperature[j],axis=0)

40.0


array([0.0030348 , 0.00374231, 0.00380715, ..., 0.0047931 , 0.00470008,
       0.00481023])

In [14]:
# load check
cond = 'DC11_RF19'
outfile = 'Time_and_temp_RFHEAT_'+str(cond)
with load(outfile+'.npz') as data:
    a = data['time']
    b = data['temp']
print(a)
print(b)
print(shape(a))
print(shape(b))

[0.00100062 0.00100112 0.00100162 ... 0.05099912 0.05099962 0.05100012]
[0.00331275 0.00445442 0.00304557 ... 0.00423682 0.00426452 0.00424611]
(100000,)
(100000,)


In [81]:
whos

Variable                                       Type                          Data/Info
--------------------------------------------------------------------------------------
C_e                                            float                         1.602e-19
Omega                                          float                         12880529.879718151
Path                                           type                          <class 'pathlib.Path'>
T_max                                          int                           50
T_thr                                          int                           1
Temperature                                    list                          n=24
Time                                           list                          n=24
Urf                                            list                          n=25
a                                              ndarray                       100000: 100000 elems, type `float64`, 800000 bytes (781.25 kb

# Work with data

In [83]:
file_path = load_file_GUI('/home/adrian/Documents/Programmes/Python')
print(file_path)

('/home/adrian/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_DC11_RF00.npz', '/home/adrian/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_DC11_RF01.npz', '/home/adrian/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_DC11_RF02.npz', '/home/adrian/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_DC11_RF03.npz', '/home/adrian/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_DC11_RF04.npz', '/home/adrian/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_DC11_RF05.npz', '/home/adrian/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_DC11_RF06.npz', '/home/adrian/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_DC11_RF07.npz', '/home/adrian/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_DC11_RF08.npz', '/home/adrian/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_DC11_RF11.npz', '/home/adrian/Documents/Progr

In [85]:
# load check
# cond = 'DC11_RF00'
# outfile = 'Time_and_temp_RFHEAT_'+str(cond)
Time = []
Temperature = []
for j,k in enumerate(file_path):
    with load(k) as data:
        Time.append( data['time'] )
        Temperature.append( data['temp'] )
    print(j,k)
print(shape(Time))
print(shape(Temperature))

0 /home/adrian/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_DC11_RF00.npz
1 /home/adrian/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_DC11_RF01.npz
2 /home/adrian/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_DC11_RF02.npz
3 /home/adrian/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_DC11_RF03.npz
4 /home/adrian/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_DC11_RF04.npz
5 /home/adrian/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_DC11_RF05.npz
6 /home/adrian/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_DC11_RF06.npz
7 /home/adrian/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_DC11_RF07.npz
8 /home/adrian/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_DC11_RF08.npz
9 /home/adrian/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_DC11_RF11.npz
10 /home/adrian/Documents/Programmes/Pyt

In [229]:
Urf = list(linspace(60,68,9)) \
    + [62.2, 62.4, 62.6, 62.8, 63.2, 63.4, 63.6, 63.8] \
    + list(linspace(40,55,4)) \
    + list(linspace(70,85,4))
print(Urf)
print(len(Urf))

[60.0, 61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68.0, 62.2, 62.4, 62.6, 62.8, 63.2, 63.4, 63.6, 63.8, 40.0, 45.0, 50.0, 55.0, 70.0, 75.0, 80.0, 85.0]
25


In [701]:
imax = len(Urf)
print('imax',imax)

T_max = 50
T_thr = 1

figure(10); clf();
i = 19
print(i,Urf[i])
#     subplot(1,9,i+1)
semilogy(Time[i]*1e3,Temperature[i],label=' Vrf = '+str(Urf[i]))
#     semilogy(time[i][:]*1e3,Temp[i][:],label=' Vrf = '+str(Urf[i]))
#         semilogy(time[k_thr],Temp[i,j,k_thr],'o')
grid()
legend(fontsize=12)

imax 25
19 50.0


In [28]:
imax = len(Urf)
print('imax',imax)

T_max = 50
T_thr = 1

figure(1); clf();
for i in range(0,imax-1):
    print(i)
    k_max = argmin(abs(Temperature[i] - T_max))
    k_thr = argmin(abs(Temperature[i] - T_thr))
    k_min = k_thr - (k_max - k_thr)
    if k_min < 0:
        k_min = 0

    i_aux = where(diff(abs(diff(Temperature[i][k_thr:k_max])))>2)
    if size(i_aux) != 0 :
        if (i_aux[0][0]+k_thr < k_max):
            k_max = i_aux[0][0]+k_thr

#     subplot(1,9,i+1)
    subplot(imax//3,4,i+1)
    semilogy(Time[i][k_min:k_max]*1e3,Temperature[i][k_min:k_max],label=' Vrf = '+str(Urf[i]))
#     semilogy(time[i][:]*1e3,Temp[i][:],label=' Vrf = '+str(Urf[i]))
#         semilogy(time[k_thr],Temp[i,j,k_thr],'o')
    grid()
    legend(fontsize=12)

imax 27
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25


IndexError: list index out of range

In [104]:
def func2(t, t0, A, B,  C, D):
    return A*(t-t0) / (C+(t-t0)**2)**D + B

def dfunc2_dt(t, t0, A, B,  C, D):
    return 10**(A*(C + (t - t0)**2)**(-D)*(t - t0) + B)*(-A*D*(C + (t - t0)**2)**(-D)*(t - t0)*(2*t - 2*t0)/(C + (t - t0)**2) + A*(C + (t - t0)**2)**(-D))*log(10)

In [704]:
T_max = 200
T_thr = 1
fit_param = zeros((imax,5))
fit_pcov  = zeros((imax,5,5))
my_order = []

figure(1); clf()
for i in range(0,imax):
    print('<<<',i,'>>>')
    k_max = argmin(abs(Temperature[i] - T_max))
    k_thr = argmin(abs(Temperature[i] - T_thr))

    i_aux = where(diff(abs(diff(Temperature[i][k_thr:k_max])))>2)
    if size(i_aux) != 0 :
        if (i_aux[0][0]+k_thr < k_max):
            k_max = i_aux[0][0]+k_thr

    k_min = k_thr - int(2*(k_max - k_thr))
    if k_min < 0: k_min = 0
#         print(k_min)

    x  = Time[i][k_min:k_max]
    y  = Temperature[i][k_min:k_max]
    x0 = Time[i][k_thr]

    A0  = 1
    B0  = log10(Temperature[i][k_thr])
    C0  = 1.0e-8
    D0  = 0.5
    p0 = array([x0, A0, B0,  C0, D0])

    try:
        popt, pcov = curve_fit(func2, x, log10(y), p0, maxfev= 500000)
        fit_param[i,:] = popt[0], popt[1], popt[2], popt[3], popt[4]
        fit_pcov[i,:]  = pcov

        y_fit = func2(x,popt[0], popt[1], popt[2], popt[3], popt[4])
        y_fit = 10**y_fit

        subplot(imax//3,4,i+1)
        semilogy(x*1e3, y    ,label=' Vrf = '+str(Urf[i]))
        semilogy(x*1e3, y_fit,label='Fit')
        grid()
        legend(fontsize=10)
        # select cases
        # non zero fit value
        my_order.append(i)
    except TypeError:
        print('TypeError for ',Urf[i])

<<< 0 >>>
<<< 1 >>>
<<< 2 >>>
<<< 3 >>>
<<< 4 >>>
<<< 5 >>>


/home/adrian/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in power
  


<<< 6 >>>
<<< 7 >>>
<<< 8 >>>
<<< 9 >>>
<<< 10 >>>
<<< 11 >>>
<<< 12 >>>
<<< 13 >>>
<<< 14 >>>
<<< 15 >>>
<<< 16 >>>
<<< 17 >>>
TypeError for  40.0
<<< 18 >>>
TypeError for  45.0
<<< 19 >>>
TypeError for  50.0
<<< 20 >>>
TypeError for  55.0
<<< 21 >>>
<<< 22 >>>
<<< 23 >>>
<<< 24 >>>
TypeError for  85.0


/home/adrian/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in true_divide
  


In [705]:
# select only Urf values
# for which non zero fit
Urf_order = [Urf[x] for w,x in enumerate(my_order)]
# sort my_order
# so Urf is ascending
zipped_lists = zip(Urf_order, my_order)
sorted_zipped_lists = sorted(zipped_lists)
for i,j in enumerate(sorted_zipped_lists):
    my_order[i] = j[1]
# delete manually
# other innapropriate values
my_order.remove(23)
Urf_order.remove(80.0)
my_order.remove(22)
Urf_order.remove(75.0)
Urf_order = list(sort([Urf[x] for w,x in enumerate(my_order)]))
print(Urf_order)
print(my_order)

# resort all variables
# in order so
# Urf ascending
imax_order = len(Urf_order)
fit_param_order = zeros((imax_order,5))
fit_pcov_order  = zeros((imax_order,5,5))
Time_order = []
Temperature_order = []
for i,j in enumerate(my_order):
    fit_param_order[i,:] = fit_param[j,:]
    fit_pcov_order[i,:]  = fit_pcov[j,:]
    Time_order.append(Time[j])
    Temperature_order.append(Temperature[j])

[60.0, 61.0, 62.0, 62.2, 62.4, 62.6, 62.8, 63.0, 63.2, 63.4, 63.6, 63.8, 64.0, 65.0, 66.0, 67.0, 68.0, 70.0]
[0, 1, 2, 9, 10, 11, 12, 3, 13, 14, 15, 16, 4, 5, 6, 7, 8, 21]


In [725]:
# draw one temp and fit
figure(3); clf()
cm = pylab.get_cmap('jet')

i = 9
print(Urf_order[i])
# print(fit_pcov_order[i,:])
y_fit = func2(Time_order[i],
              fit_param_order[i,0],
              fit_param_order[i,1],
              fit_param_order[i,2],
              fit_param_order[i,3],
              fit_param_order[i,4])
y_fit = 10**y_fit
subplot(121)
plot((Time_order[i])*1e3,Temperature_order[i],color='r')
plot((Time_order[i])*1e3,y_fit,'--',color='k')
xlabel('time [ms]')
ylabel('T [K]')
grid()

# xlim(3.6,3.7)
# ylim(-1,10)
subplot(122)
semilogy((Time_order[i])*1e3,Temperature_order[i],color='r')
semilogy((Time_order[i])*1e3,y_fit,'--',color='k')
xlabel('time [ms]')
grid()
tight_layout()

63.4


In [706]:
str_labels = ['t0', 'A', 'B', 'C', 'D']
figure('Param'); clf();
for i in range(0,5):
    subplot(1,5,i+1)
    if i == 2:
        plot(Urf_order,10**(fit_param_order[:,i]),'o-', label=str_labels[i])
    else:
        plot(Urf_order,(fit_param_order[:,i]),'o-', label=str_labels[i])
    xlabel('Urf')
    grid()
    legend()

In [707]:
figure(29); clf()
cm = pylab.get_cmap('jet')

i = 0
for i in range(0,imax_order):
    t0, A, B, C, D = fit_param_order[i,:]
    y  = func2(Time_order[i],t0, A, B, C, D);
    y  = 10**y
    dy = dfunc2_dt(Time_order[i], t0, A, B,  C, D)
    semilogy((Time_order[i]-t0)*1e3,dy/y,color=cm(i/imax))
    
i = 1
for i in range(0,imax_order):
    t0, A, B, C, D = fit_param_order[i,:]
    y  = func2(Time_order[i],t0, A, B, C, D);
    y  = 10**y
    dy = dfunc2_dt(Time_order[i], t0, A, B,  C, D)
    semilogy((Time_order[i]-t0)*1e3,dy/y,'--',color=cm(i/imax))

In [716]:
xa = -0.6
xb = 0.6
ya = 6e-6
yb = 1e0
figure(30); clf()
ax = subplot(111)
for i in range(0,imax_order):
    t0, A, B, C, D = fit_param_order[i,:]
    y  = func2(Time_order[i],t0, A, B, C, D);
    y  = 10**y
    dy = dfunc2_dt(Time_order[i], t0, A, B,  C, D)
    ax.semilogy((Time_order[i]-t0)*1e3,dy*1e-6,label=str(Urf[i]),color=cm(i/imax_order))
    ax.set_xlabel('$t-t_0$ [ms]')
    ax.set_ylabel('Heating rate [K/µs]')
#     scatter(t0,4e5,color=cm(i/imax),marker='+')
#     semilogy(y,dy*1e-3,label=str(Urf[i]),color=cm(i/imax))
    ax.set_xlim([xa,xb])
    ax.set_ylim([ya,yb])
    grid()
    
locmaj = matplotlib.ticker.LogLocator(base=10,numticks=12) 
ax.yaxis.set_major_locator(locmaj)
locmin = matplotlib.ticker.LogLocator(base=10.0,subs=(0.2,0.4,0.6,0.8),numticks=12)
ax.yaxis.set_minor_locator(locmin)
ax.yaxis.set_minor_formatter(matplotlib.ticker.NullFormatter())
ax.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.1))
ax.grid(b=True, which='major', linestyle='-')
ax.grid(b=True, which='minor', linestyle='--')
tight_layout()

In [298]:
xa = -0.05
xb = 0.05
ya = 1e-2
yb = 6e-1
figure(31); clf()
ax = subplot(111)
for i in range(0,imax_order):
    t0, A, B, C, D = fit_param_order[i,:]
    y  = func2(Time_order[i],t0, A, B, C, D);
    y  = 10**y
    dy = dfunc2_dt(Time_order[i], t0, A, B,  C, D)
    ax.semilogy((Time_order[i]-t0)*1e3,dy*1e-6/y,label=str(Urf[i]),color=cm(i/imax_order))
    ax.set_xlabel('$t-t_0$ [ms]')
    ax.set_ylabel('Heating rate / T\n[1/µs]')
#     scatter(t0,4e5,color=cm(i/imax),marker='+')
#     semilogy(y,dy*1e-3,label=str(Urf[i]),color=cm(i/imax))
    ax.set_xlim([xa,xb])
    ax.set_ylim([ya,yb])
    grid()
    
locmaj = matplotlib.ticker.LogLocator(base=10,numticks=12) 
ax.yaxis.set_major_locator(locmaj)
locmin = matplotlib.ticker.LogLocator(base=10.0,subs=(0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9),numticks=12)
ax.yaxis.set_minor_locator(locmin)
ax.yaxis.set_minor_formatter(matplotlib.ticker.NullFormatter())
ax.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.02))
ax.grid(b=True, which='major', linestyle='-')
ax.grid(b=True, which='minor', linestyle='--')
tight_layout()

# t0 and T as a function of Urf

In [728]:
t0    = zeros(imax_order)
y_t0  = zeros(imax_order)
dy_t0 = zeros(imax_order)
for i in range(0, imax_order):
    t0[i], A, B, C, D = fit_param_order[i,:]
    y  = func2(t0[i],t0[i], A, B, C, D);
    y_t0[i]  = 10**y
    dy_t0[i] = dfunc2_dt(t0[i], t0[i], A, B,  C, D)

In [729]:
def my_lin(x,a,b):
    return a*x+b
def my_power_law(x,a,b):
    return a*(x**-b)

In [730]:
# linear plot
figure(321); clf();
subplot(111)
plot(Urf_order,t0*1e3,ls='-',marker='+',mew=5,ms=10)
grid()
xlabel('$U_{rf}$ [V]')
ylabel('$t_0$ [ms]') # '$T(t_0) / t_0$ [K/s]'

tight_layout()

In [731]:
# log plot
figure(321); clf();
subplot(111)
loglog(Urf_order,t0*1e3,ls='-',marker='+',mew=5,ms=10)
grid()
xlabel('$U_{rf}$ [V]')
ylabel('$t_0$ [ms]') # '$T(t_0) / t_0$ [K/s]'

tight_layout()

In [763]:
# find the coefficient of power law
xx = Urf_order
yy = (t0)*1e3 # (yy-min(t0))*1e3
# xx = xx[1:]
# yy = -yy[1:]
popt, pcov = curve_fit(my_lin,log(xx),log(yy))

k_fromlog = popt[0]
B_fromlog = popt[1]
print(k_fromlog,B_fromlog)
# plot data
# linearized with corresponding power -popt[0]

figure(3210); clf();
ax = subplot(111)
ax.plot(xx,yy,ls='',marker='+',mew=3,ms=10)
ax.grid()
# ax.set_xlabel(f'$U_{{rf}}^{{{popt[0]:.2e}}}$ [V]')
ax.set_xlabel('$U_{rf}$ [V]')
ax.set_ylabel('$t_0$ [ms]') # '$T(t_0) / t_0$ [K/s]'
# ax.errorbar(popt[0],yy,yerr = 1e-2,ls='') # sqrt(fit_pcov_order[:,0,0])*1e7

# fit the power law
popt, pcov = curve_fit(my_power_law,xx,yy,p0=[1e23,12.5],maxfev=2500)
print(popt)
xx_fit = linspace(59.8,71,100)
ax.plot(xx_fit,my_power_law(array(xx_fit),popt[0],popt[1]),
    color='r',ls='--',label='Direct pow. Law fit')
ax.plot(xx_fit,my_power_law(array(xx_fit),B_fromlog,k_fromlog),
    color='b',ls=':',label='From log')
# ax.plot(linspace(0.5,72,1000),my_power_law(linspace(0.5,72,1000),1e23,12.5),
#     color='g',ls='--',label='test pow. Law')

ax.legend()
# ax.text(4.15,2,f'$a={popt[0]:.2f}, b={popt[1]:.2f}$',
#             color='xkcd:black',fontsize='20',weight="bold",
#             ha='left',va='center')

# ax.set_xlim(-2,72)
# ax.set_ylim(-1,18)
tight_layout()

-15.233803215236648 64.74541613417573
[5.23738105e+44 2.44715990e+01]


In [661]:
10**B_fromlog

5.564371713038298e+64

In [ ]:
# find the coefficient of power law
xx = Urf_order - Urf_order[0]
yy = (t0)*1e3 # (yy-min(t0))*1e3
xx = xx[1:]
yy = -yy[1:]

popt, pcov = curve_fit(my_lin,log(xx),log(yy))
print(popt[0],10**(popt[1]))

# plot data
# linearized with corresponding power -popt[0]

figure(3210); clf();
ax = subplot(111)
ax.plot(xx**popt[0],yy,ls='',marker='+',mew=3,ms=10)
ax.grid()
ax.set_xlabel(f'$U_{{rf}}^{{{popt[0]:.2e}}}$ [V]')
ax.set_ylabel('$t_0$ [ms]') # '$T(t_0) / t_0$ [K/s]'
print(fit_pcov_order[5,:])
ax.errorbar(xx**popt[0],yy,yerr = 1e-2,ls='') # sqrt(fit_pcov_order[:,0,0])*1e7

# fit the power law
popt, pcov = curve_fit(my_lin,xx**popt[0],yy,p0=[1e25,0])
xx_fit = linspace(0.7e-28,8.3e-28,100)
ax.plot(xx_fit,my_lin(array(xx_fit),popt[0],popt[1]),
    color='k',ls='--')


# ax.text(4.15,2,f'$a={popt[0]:.2f}, b={popt[1]:.2f}$',
#             color='xkcd:black',fontsize='20',weight="bold",
#             ha='left',va='center')

tight_layout()

In [526]:
yy_moy = mean(yy)
residual = 0
tot = 0
for i,j in enumerate(yy):
    index_for_res = Urf[i]**
    residual += yy - my_lin(array(xx_fit)[10],popt[0],popt[1])
    tot += yy - yy_moy
    R_param = residual/tot

1.5680162450037478e-07

In [304]:
# 1/sqrt() plot
figure(321); clf();
ax = subplot(111)
ax.plot(Urf_order,(t0*1e3)**-0.5,ls='-',marker='+',mew=5,ms=10)
ax.grid()
ax.set_xlabel('$U_{rf}$ [V]')
ax.set_ylabel('$1/\sqrt{t_0}$ [ms]') # '$T(t_0) / t_0$ [K/s]'

popt, pcov = curve_fit(my_lin,Urf_order,(t0*1e3)**-0.5)
ax.plot(Urf_order,my_lin(array(Urf_order),popt[0],popt[1]),
    color='k',ls='--')

ax.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=1))
ax.yaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.1))

ax.text(66.3,0.75,f'$1/\sqrt{{t_0}} = {popt[0]:.2e} U_{{rf}} + {popt[1]:.2f}$',
            color='xkcd:black',fontsize='20',weight="bold",
            ha='right',va='center')

tight_layout()

In [305]:
figure(322); clf();
ax = subplot(111)
ax.plot(Urf_order,y_t0,ls='-',marker='+',mew=5,ms=10)
ax.grid()
ax.set_xlabel('$U_{rf}$ [V]')
ax.set_ylabel('$T(t_0)$ [K]') # '$T(t_0) / t_0$ [K/s]'

popt, pcov = curve_fit(my_lin,Urf_order,y_t0)
ax.plot(Urf_order,my_lin(array(Urf_order),popt[0],popt[1]),
    color='k',ls='--')

ax.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=1))
ax.yaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.1))

ax.text(64,0.815,f'$T(t_0) = {popt[0]:.2e} U_{{rf}} + {popt[1]:.2f}$',
            color='xkcd:black',fontsize='20',weight="bold",
            ha='left',va='center')

tight_layout()

# identique à 10**B

In [306]:
figure(323); clf();
subplot(111)
plot(Urf_order,y_t0,ls='-',marker='+',mew=5,ms=10)
grid()
xlabel('$U_{rf}$ [V]')
ylabel('$T(t_0)$ [K]') # '$T(t_0) / t_0$ [K/s]'

tight_layout()

# identique à 10**B

In [ ]:
savefig('T_t0_Udc.png')

In [79]:
figure(33); clf();
subplot(121)
title('Vrf = '+str(Urf[2]))
plot(Udc,t0_v,'o')
grid()
xlabel('Udc[V]')
ylabel('T[t = t0] / t0 [K/s]')

subplot(122)
title('Vrf = '+str(Urf[3]))
plot(Udc,t1_v,'o')
grid()
xlabel('Udc[V]')
ylabel('T[t = t0] / t0 [K/s]')


NameError: name 'Udc' is not defined

In [223]:
figure(33); clf();
subplot(121)
title('Vrf = '+str(Urf[2]))
plot(Udc,y_t0,'o')
grid()
xlabel('Udc[V]')
ylabel('T[t = t0]')

subplot(122)
title('Vrf = '+str(Urf[3]))
plot(Udc,y_t1,'o')
grid()
xlabel('Udc[V]')
ylabel('T[t = t0]')


Text(0, 0.5, 'T[t = t0] / t0 [K/s]')

In [80]:
figure(31); clf()

i = 0
for j in range(0,jmax):
    t0, A, B, C, D = fit_param[i,j,:]
    y  = func2(time,t0, A, B, C, D);
    y  = 10**y
    dy = dfunc2_dt(time, t0, A, B,  C, D)
    semilogy(time-t0,y)
    
i = 1
for j in range(0,jmax):
    t0, A, B, C, D = fit_param[i,j,:]
    y  = func2(time,t0, A, B, C, D);
    y  = 10**y
    dy = dfunc2_dt(time, t0, A, B,  C, D)
    title('Vrf = '+str(Urf[i+2]))
    semilogy(time-t0,y,'--')

NameError: name 'jmax' is not defined